In [ ]:
import os
from pyspark.sql import SparkSession, functions as F, types as T

#Конфигурация 
HDFS_NAMENODE = "rc1a-dataproc-m-0tvi3u0a47j8dj6t.mdb.yandexcloud.net:8020"
HDFS_INPUT_DIR = "/user/ubuntu/data"
HDFS_OUTPUT_DIR = "/user/ubuntu/clean/transactions_parquet"
S3_OUTPUT_PATH = "s3a://otus-bucket-20251311-b1ghiv85eubrk846dis6/clean/transactions_parquet"
S3_ACCESS_KEY = "-"
S3_SECRET_KEY = "-"

# Инициализация Spark с подавлением спилл предупреждений
spark = SparkSession.builder \
    .appName("clean_transactions") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.access.key", S3_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", S3_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.endpoint", "storage.yandexcloud.net") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "400") \
    .config("spark.driver.memory", "8g") \
    .config("spark.driver.maxResultSize", "1g") \
    .config("spark.executor.memory", "10g") \
    .config("spark.executor.memoryOverhead", "3g") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.1") \
    .config("spark.sql.adaptive.advisoryPartitionSizeInBytes", "64MB") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.sql.adaptive.localShuffleReader.enabled", "true") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
    .config("spark.sql.adaptive.maxRecordsPerFile", "500000") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "1024m") \
    .config("spark.sql.sources.bucketing.enabled", "false") \
    .config("spark.sql.execution.sortBeforeRepartition", "false") \
    .config("spark.sql.execution.useObjectHashAggregateExec", "false") \
    .config("spark.sql.execution.fastHashAggregateRowMaxCapacityBit", "16") \
    .getOrCreate()


spark.sparkContext.setLogLevel("ERROR")

# Схема данных
schema = T.StructType([
    T.StructField("transaction_id", T.IntegerType(), nullable=False),
    T.StructField("tx_datetime", T.StringType(), nullable=False),
    T.StructField("customer_id", T.IntegerType(), nullable=False),
    T.StructField("terminal_id", T.IntegerType(), nullable=False),
    T.StructField("tx_amount", T.DoubleType(), nullable=False),
    T.StructField("tx_time_seconds", T.IntegerType(), nullable=False),
    T.StructField("tx_time_days", T.IntegerType(), nullable=False),
    T.StructField("tx_fraud", T.IntegerType(), nullable=False),
    T.StructField("tx_fraud_scenario", T.IntegerType(), nullable=False),
])

# Чтение данных из HDFS
input_path = f"hdfs://{HDFS_NAMENODE}{HDFS_INPUT_DIR}/*"
df_raw = spark.read.option("header", "false").option("sep", ",").schema(schema).csv(input_path)

# Сохраняем исходное количество строк
total_raw = df_raw.count()
print(f"✓ Загружено: {total_raw:,} строк из HDFS")

✓ Загружено: 46,988,419 строк из HDFS


In [3]:
# === 2. Анализ и очистка данных ===
print("\n=== АНАЛИЗ ПРОБЛЕМ КАЧЕСТВА ===")

# Быстрая проверка основных проблем
print(f"1. ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ: {df_raw.filter(F.col('tx_fraud').isNull()).count()}")
print(f"2. НЕКОРРЕКТНЫЕ СУММЫ: {df_raw.filter(F.col('tx_amount') <= 0).count()}")
print(f"3. НЕКОРРЕКТНОЕ ВРЕМЯ: {df_raw.filter(F.col('tx_datetime').rlike('24:00:00')).count()}")

# Дубликаты
total_records = df_raw.count()
unique_records = df_raw.select("transaction_id").distinct().count()
duplicates = total_records - unique_records
print(f"4. ДУБЛИКАТЫ: {duplicates}")

print(f"\n=== КОМПЛЕКСНАЯ ОЧИСТКА ДАННЫХ ===")

# Очистка
df = df_raw.withColumn(
    "tx_datetime",
    F.to_timestamp(
        F.regexp_replace("tx_datetime", " 24:00:00$", " 23:59:59"),
        "yyyy-MM-dd HH:mm:ss"
    )
).filter(
    (F.col("tx_fraud").isNotNull()) &
    (F.col("tx_amount") > 0) & 
    (F.col("tx_amount") < 1e7) &
    (F.col("tx_time_seconds") >= 0) &
    (F.col("tx_time_days") >= 0) &
    (F.col("tx_datetime").isNotNull())
).dropDuplicates(["transaction_id"])

# Сохраняем количество строк после очистки
total_clean = df.count()
removed = total_raw - total_clean
removed_pct = (removed / total_raw) * 100 if total_raw > 0 else 0

print(f"✓ Очищено: {total_clean:,} строк ({100*total_clean/total_raw:.2f}%)")
print(f"✓ Удалено: {removed:,} строк ({removed_pct:.2f}%)")

# Освобождаем память ПОСЛЕ всех операций с df  
df.unpersist()


=== АНАЛИЗ ПРОБЛЕМ КАЧЕСТВА ===


1. ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ: 1


2. НЕКОРРЕКТНЫЕ СУММЫ: 884


3. НЕКОРРЕКТНОЕ ВРЕМЯ: 100


4. ДУБЛИКАТЫ: 181

=== КОМПЛЕКСНАЯ ОЧИСТКА ДАННЫХ ===


✓ Очищено: 46,987,353 строк (100.00%)
✓ Удалено: 1,066 строк (0.00%)


DataFrame[transaction_id: int, tx_datetime: timestamp, customer_id: int, terminal_id: int, tx_amount: double, tx_time_seconds: int, tx_time_days: int, tx_fraud: int, tx_fraud_scenario: int]

In [4]:
# === 3. Сохранение в HDFS и S3 ===
import subprocess

print(f"\n=== СОХРАНЕНИЕ ===")

print(f"\n📊 СТАТИСТИКА ОЧИСТКИ:")
print(f"  Исходных записей:    {total_raw:,}")
print(f"  Очищенных записей:   {total_clean:,}")
print(f"  Удалено записей:     {removed:,} ({removed_pct:.2f}%)")
print(f"  Сохранено:           {100*total_clean/total_raw:.2f}%")

hdfs_path = f"hdfs://{HDFS_NAMENODE}{HDFS_OUTPUT_DIR}"

# Подготовка директорий HDFS
subprocess.run(["hdfs", "dfs", "-mkdir", "-p", HDFS_OUTPUT_DIR], capture_output=True)
subprocess.run(["hdfs", "dfs", "-rm", "-r", HDFS_OUTPUT_DIR], capture_output=True)

# Сохранение в HDFS
print(f"\n→ Сохраняем в HDFS...")
df.coalesce(10).write.format("parquet").mode("overwrite").save(hdfs_path)
print("✓ HDFS: готово")

# Копирование в S3
print(f"→ Копируем в S3...")
result = subprocess.run(
    ["hadoop", "distcp", "-overwrite", "-m", "32", HDFS_OUTPUT_DIR, S3_OUTPUT_PATH],
    capture_output=True,
    text=True,
    timeout=1200
)
if result.returncode == 0:
    print("✓ S3: готово")
else:
    print(f"⚠ S3 копирование: {result.stderr[:200]}")


=== СОХРАНЕНИЕ ===

📊 СТАТИСТИКА ОЧИСТКИ:
  Исходных записей:    46,988,419
  Очищенных записей:   46,987,353
  Удалено записей:     1,066 (0.00%)
  Сохранено:           100.00%

→ Сохраняем в HDFS...


✓ HDFS: готово
→ Копируем в S3...
✓ S3: готово


In [5]:
# === 4. Создание standalone скрипта ===
script_path = "/home/ubuntu/clean_transactions.py"

# Формируем скрипт используя данные из Cell 1
script_content = f'''#!/usr/bin/env python3
"""Автоматическая очистка транзакционных данных для MLOps."""
import subprocess
from pyspark.sql import SparkSession, functions as F, types as T

def main():
    # Создаем Spark сессию с оптимизациями
    spark = SparkSession.builder \\
        .appName("clean_transactions") \\
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \\
        .config("spark.hadoop.fs.s3a.access.key", "{S3_ACCESS_KEY}") \\
        .config("spark.hadoop.fs.s3a.secret.key", "{S3_SECRET_KEY}") \\
        .config("spark.hadoop.fs.s3a.endpoint", "storage.yandexcloud.net") \\
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \\
        .config("spark.sql.adaptive.enabled", "true") \\
        .config("spark.sql.shuffle.partitions", "400") \\
        .config("spark.driver.memory", "8g") \\
        .config("spark.executor.memory", "10g") \\
        .config("spark.executor.memoryOverhead", "3g") \\
        .config("spark.memory.fraction", "0.8") \\
        .config("spark.memory.storageFraction", "0.1") \\
        .getOrCreate()
    
    # Подавляем WARN сообщения
    spark.sparkContext.setLogLevel("ERROR")
    
    # Конфигурация (автоматически взята из notebook)
    HDFS_NAMENODE = "{HDFS_NAMENODE}"
    HDFS_INPUT_DIR = "{HDFS_INPUT_DIR}"
    HDFS_OUTPUT_DIR = "{HDFS_OUTPUT_DIR}"
    S3_OUTPUT_PATH = "{S3_OUTPUT_PATH}"

    # Схема данных
    schema = T.StructType([
        T.StructField("transaction_id", T.IntegerType(), nullable=False),
        T.StructField("tx_datetime", T.StringType(), nullable=False),
        T.StructField("customer_id", T.IntegerType(), nullable=False),
        T.StructField("terminal_id", T.IntegerType(), nullable=False),
        T.StructField("tx_amount", T.DoubleType(), nullable=False),
        T.StructField("tx_time_seconds", T.IntegerType(), nullable=False),
        T.StructField("tx_time_days", T.IntegerType(), nullable=False),
        T.StructField("tx_fraud", T.IntegerType(), nullable=False),
        T.StructField("tx_fraud_scenario", T.IntegerType(), nullable=False),
    ])

    print("=== НАЧАЛО ОЧИСТКИ ДАННЫХ ===")
    
    # 1. Чтение данных из HDFS
    input_path = f"hdfs://{{HDFS_NAMENODE}}{{HDFS_INPUT_DIR}}/*"
    df_raw = spark.read.option("header", "false").option("sep", ",").schema(schema).csv(input_path)
    
    total_raw = df_raw.count()
    print(f"✓ Загружено: {{total_raw:,}} строк из HDFS")

    # 2. Быстрый анализ качества данных
    print("\\n=== АНАЛИЗ КАЧЕСТВА ===")
    null_fraud = df_raw.filter(F.col('tx_fraud').isNull()).count()
    invalid_amount = df_raw.filter(F.col('tx_amount') <= 0).count()
    invalid_time = df_raw.filter(F.col('tx_datetime').rlike('24:00:00')).count()
    
    total_records = df_raw.count()
    unique_records = df_raw.select("transaction_id").distinct().count()
    duplicates = total_records - unique_records
    
    print(f"• NULL значения tx_fraud: {{null_fraud}}")
    print(f"• Некорректные суммы: {{invalid_amount}}")
    print(f"• Некорректное время: {{invalid_time}}")
    print(f"• Дубликаты: {{duplicates}}")

    # 3. Очистка данных
    print("\\n=== ОЧИСТКА ДАННЫХ ===")
    df_clean = df_raw.withColumn(
        "tx_datetime",
        F.to_timestamp(
            F.regexp_replace("tx_datetime", " 24:00:00$", " 23:59:59"),
            "yyyy-MM-dd HH:mm:ss"
        )
    ).filter(
        (F.col("tx_fraud").isNotNull()) &
        (F.col("tx_amount") > 0) & (F.col("tx_amount") < 1e7) &
        (F.col("tx_time_seconds") >= 0) &
        (F.col("tx_time_days") >= 0) &
        (F.col("tx_datetime").isNotNull())
    ).dropDuplicates(["transaction_id"])
    
    total_clean = df_clean.count()
    removed = total_raw - total_clean
    removed_pct = (removed / total_raw) * 100 if total_raw > 0 else 0
    
    print(f"✓ Очищено: {{total_clean:,}} строк ({{100*total_clean/total_raw:.2f}}%)")
    print(f"✓ Удалено: {{removed:,}} строк ({{removed_pct:.2f}}%)")

    # 4. Сохранение в HDFS
    print("\\n=== СОХРАНЕНИЕ ===")
    hdfs_path = f"hdfs://{{HDFS_NAMENODE}}{{HDFS_OUTPUT_DIR}}"
    
    # Подготовка директорий
    subprocess.run(["hdfs", "dfs", "-mkdir", "-p", HDFS_OUTPUT_DIR], capture_output=True)
    subprocess.run(["hdfs", "dfs", "-rm", "-r", HDFS_OUTPUT_DIR], capture_output=True)
    
    print("→ Сохраняем в HDFS...")
    df_clean.coalesce(10).write.format("parquet").mode("overwrite").save(hdfs_path)
    print("✓ HDFS: готово")
    
    # 5. Копирование в S3
    print("→ Копируем в S3...")
    result = subprocess.run(
        ["hadoop", "distcp", "-overwrite", "-m", "32", HDFS_OUTPUT_DIR, S3_OUTPUT_PATH],
        capture_output=True,
        text=True,
        timeout=1800
    )
    if result.returncode == 0:
        print("✓ S3: готово")
    else:
        print(f"⚠ S3 ошибка: {{result.stderr[:200]}}")
    
    print(f"\\n=== ФИНАЛЬНАЯ СТАТИСТИКА ===")
    print(f"• Обработано: {{total_raw:,}} → {{total_clean:,}} записей")
    print(f"• HDFS путь: {{HDFS_OUTPUT_DIR}}")
    print(f"• S3 путь: {{S3_OUTPUT_PATH}}")
    print("=== ЗАВЕРШЕНО ===")
    
    spark.stop()

if __name__ == "__main__":
    main()
'''

# Создаём скрипт
with open(script_path, 'w') as f:
    f.write(script_content)

# Делаем исполняемым
import subprocess
subprocess.run(['chmod', '+x', script_path])
    
print(f"✅ Скрипт создан: {script_path}")
print(f"📋 Конфигурация автоматически взята из Cell 1:")
print(f"   • HDFS: {HDFS_NAMENODE}")
print(f"   • Bucket: {S3_OUTPUT_PATH}")
print(f"   • Keys: {S3_ACCESS_KEY[:10]}...") 
print(f"\n🚀 Запуск:")
print(f"   spark-submit {script_path}")

✅ Скрипт создан: /home/ubuntu/clean_transactions.py
📋 Конфигурация автоматически взята из Cell 1:
   • HDFS: rc1a-dataproc-m-0tvi3u0a47j8dj6t.mdb.yandexcloud.net:8020
   • Bucket: s3a://otus-bucket-20251311-b1ghiv85eubrk846dis6/clean/transactions_parquet
   • Keys: YCAJEBaG2V...

🚀 Запуск:
   spark-submit /home/ubuntu/clean_transactions.py
